In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [3]:
import galois
import numpy as np
import time
import os
from scipy import stats

In [4]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(8)

In [5]:
#definisco una matrice k x (n-k)
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [6]:
#questa è con l'esempio di 4.3
A = field([[6, 6, 3, 0],
           [7, 5, 0, 7],
           [5, 0, 6, 1]])

In [14]:
A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)

[[2 2 7 1]
 [7 6 0 0]
 [4 2 7 2]]


In [15]:
matrix_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrix_bin1 = field(matrix_bin1)

matrix_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrix_bin2 = field(matrix_bin2)

In [16]:
A_primo = field(matrix_bin1 @ A @ matrix_bin2)
print(A_primo)

[[4 2 2 7]
 [2 1 2 7]
 [7 0 6 0]]


In [8]:
#number of iteration for Montecarlo simulatiom
num_iterazioni = 2500

In [55]:
# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix):
    operations = 0
    vector_of_counts = []
    matrix_int = matrix.view(np.ndarray)  
    for row in matrix_int:
        counts = [0] * field.order 
        for value in row:
            counts[value] += 1
            operations += 1
        #print(counts)
        vector_of_counts.append(counts)
        #print(vector_of_counts)
    #print(vector_of_counts)
    vector_of_counts = np.array(vector_of_counts)
    # Ordina la lista di dizionari usando la chiave personalizzata
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(field.order-1, -1, -1)])[::-1]

    operations1 = 0
    vector_of_counts1 = []
    matrix_int = matrix.view(np.ndarray)  
    for row in matrix_int:
        counts = [0] * 3
        for value in row:
            #print(value)
            #print(value < 3)
            if value < 3:
                counts[value] += 1
                operations1 += 1
        #print(counts)
        vector_of_counts1.append(counts)
        #print(vector_of_counts1)
    vector_of_counts1 = np.array(vector_of_counts1)
    # Ordina la lista di dizionari usando la chiave personalizzata
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(3-1, -1, -1)])[::-1]
    print("Ecco qui\n",matrix[sorted_index], matrix[sorted_index1])
    
    return matrix[sorted_index]

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    return sorted_matrix, num_swap_columns

def algorithm_test(matrix1, matrix2):
    row1_sort = order_row(matrix1)
    print("Ordinato\n", row1_sort)
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    print(matrix1_sorted)

    row2_sort = order_row(matrix2)
    print(row2_sort)
    matrix2_sorted = sort_by_column(row2_sort, range(k), 0)
    print(matrix2_sorted)

    #[0] because retur an array with 2 elements
    return np.array_equal(matrix1_sorted[0], matrix2_sorted[0])



In [56]:
algorithm_test(A, A_primo)

Ecco qui
 [[7 6 0 0]
 [2 2 7 1]
 [4 2 7 2]] [[7 6 0 0]
 [2 2 7 1]
 [4 2 7 2]]
Ordinato
 [[7 6 0 0]
 [2 2 7 1]
 [4 2 7 2]]
Funzione ordina per colonna
(GF([[0, 0, 6, 7],
    [1, 7, 2, 2],
    [2, 7, 2, 4]], order=2^3), 4)
Ecco qui
 [[7 0 6 0]
 [2 1 2 7]
 [4 2 2 7]] [[7 0 6 0]
 [2 1 2 7]
 [4 2 2 7]]
[[7 0 6 0]
 [2 1 2 7]
 [4 2 2 7]]
Funzione ordina per colonna
(GF([[0, 0, 6, 7],
    [1, 7, 2, 2],
    [2, 7, 2, 4]], order=2^3), 3)


True